In [2]:
%%capture
%pip install llama-index
%pip install llama-index-vector-stores-qdrant
%pip install llama-index-vector-stores-chroma
%pip install llama-index chromadb
%pip install llama-index-readers-file
%pip install llama-index-llms-openai
%pip install qdrant_client
%pip install chromadb
%pip install llama-index-embeddings-fastembed
%pip install llama-index-embeddings-huggingface

In [3]:
pip install --upgrade huggingface-hub==0.23.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastembed 0.2.7 requires huggingface-hub<0.21,>=0.20, but you have huggingface-hub 0.23.0 which is incompatible.
fastembed 0.2.7 requires tokenizers<0.16,>=0.15, but you have tokenizers 0.19.1 which is incompatible.


In [1]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

# Settings.embed_model = HuggingFaceEmbedding(
#     model_name="mistralai/Mistral-7B-Instruct-v0.3"
# )
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore

from google.colab import userdata
import qdrant_client

client = qdrant_client.QdrantClient(
    url = "https://e1e16ea6-1c74-43be-a4a3-8564d87a65ed.us-east4-0.gcp.cloud.qdrant.io",
    api_key = userdata.get("QDRANT_API_KEY")
)

vector_store = QdrantVectorStore(client=client, collection_name="fashion_data2")
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, embed_model = embed_model)

In [4]:
%pip install llama-index-llms-mistralai

In [5]:
from google.colab import userdata

In [6]:
from llama_index.llms.mistralai import MistralAI

Settings.llm = MistralAI(api_key=userdata.get('MISTRAL_API_KEY'))


In [8]:
from IPython.display import Markdown, display

In [9]:
query_engine = index.as_query_engine()
res = query_engine.query("Suggest a good jeans for my sister. Give all product details in the json format")
display(Markdown(f"<b>{res}</b>"))

<b>{
  "Product1": {
    "BrandName": "all about you",
    "Category": "Western jeans",
    "Gender": "Women",
    "Description": "This product offers a navy blue super skinny fit high rise stretchable jeans for women.",
    "Price": "nan",
    "SizeOption": ["26", "28", "30", "32", "34"],
    "Ratings": "4.0"
  },
  "Product2": {
    "BrandName": "all about you",
    "Category": "Western jeans",
    "Gender": "Women",
    "Description": "Experience the comfort of blue skinny fit stretchable jeans with this product.",
    "Price": "nan",
    "SizeOption": ["26", "28", "30", "32", "34"],
    "Ratings": "4.2"
  }
}</b>

## RAG pipe

In [3]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor, SentenceTransformerRerank
def get_query_engine(sentence_index, similarity_top_k=6, rerank_top_n=2):
  postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
  rerank = SentenceTransformerRerank(
      top_n=rerank_top_n, model="BAAI/bge-reranker-base"
  )

  engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
  )

  return engine

ModuleNotFoundError: No module named 'llama_index'

In [10]:
query_engine = get_query_engine(sentence_index=index, similarity_top_k=6, rerank_top_n=3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [1]:
query= f"Customer query: Recommend few Bottom wears.  Give all product details including product number and specified size in the json format. Do not hallucinate"
response = query_engine.query(query)
display(Markdown(f"<b>{response}</b>"))

NameError: name 'query_engine' is not defined

In [26]:
# for i in index.as_retriever().retrieve(query):
#   print(i.text)
#   print()

**Product 13742470**.
BrandName: HERE&NOW Category: Plus Size jeans Gender: Women Description: herenow women blue skinny fit heavy fade stretchable jeans Price: nan SizeOption: 26, 28, 30, 32, 34, 36 Ratings: 4.2   



**Product 13103800**.
BrandName: HERE&NOW Category: Plus Size jeans Gender: Women Description: herenow women blue straight fit mid rise clean look side tape detail stretchable jeans Price: 839.0 SizeOption: 26, 28, 30, 32, 34, 36, 38, 40 Ratings: 4.4   





In [29]:
customer_Purchase_hist= """
HERE&NOW Plus Size jeans 34
"""

In [22]:
index.as_retriever().retrieve(query)

[NodeWithScore(node=TextNode(id_='4b40e625-13b0-4369-a38e-ab3819955dcc', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='**Product 14742868**.\nBrandName: all about you Category: Western jeans Gender: Women Description: all about you women navy blue super skinny fit high rise stretchable jeans Price: nan SizeOption: 26, 28, 30, 32, 34 Ratings: 4.0   \n\n', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.70157236),
 NodeWithScore(node=TextNode(id_='5baa4bfb-bd8c-4d6a-8d51-f2d278ea7152', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='**Product 14631764**.\nBrandName: all about you Category: Western jeans Gender: Women Description: all about you women blue slim fit stretchable jeans Price: nan SizeOption: 26, 28, 30, 32, 34 

## Customer persona

In [ ]:
import pandas as pd
import numpy as np

# Load the CSV file
file_path = 'path_to_your_file.csv'  # Replace with the path to your CSV file
df = pd.read_csv(file_path)

# Create a set of random customer IDs
num_customers = 1000  # Number of unique customers you want to create
customer_ids = [f'CUST{str(i).zfill(5)}' for i in range(1, num_customers + 1)]

# Randomly map product IDs to customer IDs
num_products = len(df)
mapping = []

# Let's decide how many products each customer will buy
np.random.seed(42)  # For reproducibility
products_per_customer = np.random.randint(1, 20, size=num_customers)

for customer_id, num_products_to_buy in zip(customer_ids, products_per_customer):
    sampled_product_ids = np.random.choice(df['product_id'], size=num_products_to_buy, replace=False)
    for product_id in sampled_product_ids:
        mapping.append({'customer_id': customer_id, 'product_id': product_id})

# Create a DataFrame from the mapping
mapping_df = pd.DataFrame(mapping)

# Save the mapping to a CSV file
mapping_df.to_csv('customer_product_mapping.csv', index=False)

print("Customer-product mapping created and saved to 'customer_product_mapping.csv'")
